In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
from datetime import date
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, './load_data')
from load import load_stats, load_history, load_td

pd.options.mode.chained_assignment = None

# Load Data

In [18]:
players = load_stats()
history = load_history()
td = load_td()

juniors = players[(players['Age'] <= timedelta(19*365)) & (players['Tournaments Played'] >= 10)]

In [19]:
active_index = td['Expiration Date'] >= '2024-09-11'
active = td[active_index]
sus = active[~(active['Member ID'].isin(players['USATT#'])) & ~(active['Member ID'].isin(history['USATT#']))]
len(sus)

51

In [20]:
def initrating(player):
    rhist = history[history['Database#'] == player['Database#']]
    return rhist.iloc[-1, 13]
sus = history[~history['USATT#'].isin(td['Member ID'])]
t = sus.apply(initrating, axis=1)
t

1344      0.0
1345      0.0
7561      0.0
12240     0.0
12241     0.0
         ... 
195350    0.0
195368    0.0
195386    0.0
195667    0.0
195678    0.0
Length: 143, dtype: float64

In [21]:
len(players)

10344

In [22]:
len(history['Database#'].unique())

10489

In [23]:
histnoplayers = history[~history['Database#'].isin(players['Database#'])]
playersnohist = players[~players['Database#'].isin(history['Database#'])]
histnotd = history[~history['USATT#'].isin(td['Member ID'])]
sus = histnoplayers[~histnoplayers['USATT#'].isin(histnotd['USATT#'])]['USATT#'].unique()

In [ ]:
len()

195715

In [25]:
print(len(histnoplayers['Database#'].unique()))
print(len(histnotd['Database#'].unique()))

145
137


In [26]:
sus = players[~players['USATT#'].isin(active['Member ID'])]
td[td['Member ID'].isin(sus['USATT#'])]

,Member ID,Last Name,First Name,Rating,State,Zip,Gender,Date of Birth,Expiration Date,Last Played Date
130,22497,Alto,Earl,2431,CA,92126,M,1988-05-04,2024-07-31,2023-04-09
336,78800,Baldonado,Jones,2183,NV,89169,M,1963-12-06,2024-06-07,2023-06-04
361,88635,Banu,George,1671,OK,73099,M,1978-02-27,2023-01-20,2022-04-02
901,4752,Chan,Oswald,1938,CA,91202,M,1961-12-21,2016-06-02,2018-03-02
994,93379,Chen,Bob,2688,CA,95121,M,1994-01-05,2024-07-03,2019-12-21
...,...,...,...,...,...,...,...,...,...,...
71057,280113,Kirsur,Nikhil,145,NaN,NaN,M,2009-10-02,2024-08-27,2024-06-22
71110,1181218,Liu,Wei,962,NaN,20148,F,1959-04-30,2024-09-04,2024-05-05
71118,1181229,Peng,Justin,963,PA,18940,M,2008-12-03,2024-09-05,2024-08-04
71119,1181230,Rao,Pranav,544,NaN,0,M,2009-06-13,2024-09-05,2023-12-03


In [27]:
def rating_check(player):
    rhist = history[history['Database#'] == player['Database#']].reset_index()
    return rhist.loc[:len(rhist)-2, 'Initial Rating'].equals(rhist.loc[1:, 'Final Rating'].reset_index(drop=True))

rcheck = players.apply(rating_check, axis=1)

In [28]:
rcheck[rcheck == True]

1        True
2        True
3        True
4        True
5        True
         ... 
10339    True
10340    True
10341    True
10342    True
10343    True
Length: 9468, dtype: bool

In [29]:
td.duplicated()[td.duplicated() == True]

Series([], dtype: bool)

In [30]:
players

,Rank,First Name,Last Name,Database#,USATT#,Location,Home Club,Tournament Rating,Last Played Tournament,League Rating,Last Played League,Membership Expiration,State,Zip,Gender,Date of Birth,Expiration Date,Max Rating,Age,Tournaments Played
0,1,Kalinikos,Kreanga,69780,14365,FN,NaN,2851,2023-10-29,0,NaT,ACTIVE,FN,0,M,1900-01-01,2024-11-01,2871.0,45586 days,3.0
1,2,Lei,Kou,165420,1165420,"Milpitas, CA",Kou Lei Table Tennis Club,2810,2024-04-07,2816,2022-08-28,ACTIVE,CA,95035,M,1987-11-20,2025-06-29,2864.0,13487 days,40.0
2,3,Enzo,Angles,169130,1169130,"Vila Nova De Gaia, FN",NaN,2808,2024-09-08,0,NaT,ACTIVE,FN,11111,M,1995-02-05,2024-09-02,NaN,10853 days,NaN
3,4,Kanak,Jha,3431,72193,"Milpitas, CA",888 Table Tennis Center,2806,2024-07-07,2737,2017-04-08,ACTIVE,CA,95035,M,2000-06-19,2100-02-28,2806.0,8892 days,67.0
4,5,Kayama,Yu,185698,1185698,NaN,NaN,2779,2024-08-04,0,NaT,ACTIVE,NaN,0,M,NaT,2025-08-08,2779.0,NaT,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10339,10477,Easton,Deng,177517,276517,"San Jose, CA",NaN,3,2023-06-03,442,2024-09-01,ACTIVE,CA,95129,M,2016-04-04,2024-11-18,3.0,3124 days,2.0
10340,10478,Steve,Roberts,178929,277929,"Walnut Creek, CA",NaN,3,2024-06-22,8,2024-02-24,ACTIVE,CA,94597,M,1962-05-02,2025-03-10,37.0,22820 days,5.0
10341,10479,Nicole,Leung,182143,1182143,NaN,NaN,2,2024-06-15,0,2023-10-14,ACTIVE,NaN,0,F,2014-05-28,2025-10-20,2.0,3801 days,1.0
10342,10480,Rounak,Sengupta,184322,1184322,San Ramon,NaN,1,2024-05-12,0,2024-04-20,ACTIVE,NaN,94583,M,2010-08-10,2025-04-16,24.0,5188 days,2.0


In [31]:
df = players['Date of Birth'].value_counts(dropna=False).sort_values(ascending=False)
print(df)

Date of Birth
NaT           245
1900-01-01    106
1997-07-07     16
1990-09-09     15
2000-01-01     10
             ... 
1997-05-10      1
1967-02-24      1
1987-03-17      1
1981-05-21      1
2012-04-07      1
Name: count, Length: 8267, dtype: int64


In [32]:
players['Date of Birth'].astype(str).str.slice(5,10).value_counts(dropna=False)

Date of Birth
         245
01-01    189
09-09     64
12-31     47
04-20     45
        ... 
02-13     16
03-09     16
01-17     16
08-09     14
02-29      8
Name: count, Length: 367, dtype: int64